### Tests
Can the LLM write decent unit tests for our functions?

There's only two functions and Ive pasted them into a py file for easy access. Lets import them.

In [2]:
from project_functions import objective, app
from project_functions import objective, app
from fastapi.testclient import TestClient
import optuna

I'll now paste the functions into LLM and ask it to create unit tests for them.

**Prompt:** use pytest to test this function

**Follow up Prompt:** The function is an objective for the Optuna library to optimize. It does not have any inputs. It creates and xgboost model and then returns the accuracy of the model. The function is fed into Optuna's study.optimize().

**Result:** 

In [ ]:
def test_objective():
    accuracy = round(
        objective(
            optuna.trial.FixedTrial({"n_estimators": 800, "max_depth": 2, "min_child_weight": 10,
                                     "learning_rate": 0.03, "gamma": 0.01, "subsample": 0.7,
                                     "colsample_bytree": 0.8, "reg_alpha": 0.01, "reg_lambda": 0.05})
                    ), 3)

    assert accuracy == 0.797

I completely rewrote this as the LLM's output was just a mess.

Optuna documentation gives good info on how to test objective functions: https://optuna.readthedocs.io/en/stable/faq.html#how-can-i-test-my-objective-functions

How about our FastAPI app?

**Prompt:** I have a function that uses pydantic's Basemodel and FastAPI to return a prediction for an xgboost model. write a test for this functions using py test.

**Follow up Prompt:** The data will be in json format. The Data model consistes of our ml model's features and dtypes. The xgboost model is a XGBClassifier object that is already in memory (its just called "model")

**Result:** 

In [ ]:
client = TestClient(app)
def test_predict():
    # Define test data
    test_data = {
       "tenure": 12,
       "MonthlyCharges": 20,
       "TotalCharges": 240,
       "TotalServices": 3,
       "gender_Male": 1,
       "InternetService_Fiber optic": 0,
       "InternetService_No": 1,
       "Contract_One year": 0,
       "Contract_Two year": 1,
       "PaymentMethod_Credit card (automatic)": 0,
       "PaymentMethod_Electronic check": 1,
       "PaymentMethod_Mailed check": 0,
       "SeniorCitizen": 0,
       "Partner": 1,
       "Dependents": 0,
       "PhoneService": 1,
       "MultipleLines": 0,
       "OnlineSecurity": 0,
       "OnlineBackup": 0,
       "DeviceProtection": 1,
       "TechSupport": 0,
       "StreamingTV": 0,
       "StreamingMovies": 0,
       "PaperlessBilling": 1,
    }
    
    # Send a POST request to the /predict endpoint
    response = client.post("/predict", json=test_data)
    
    # Check the status code
    assert response.status_code == 200
    
    # Check the returned prediction
    assert "prediction" in response.json()

Alright. This seems to align with the official documentation: https://fastapi.tiangolo.com/tutorial/testing/

it got the variable names partially wrong though (remember that i used aliases in my app). easy to change.

I suppose this is satisfactory. There could be other tests which check the actual returned values. Though this wouldnt sctrictly be testing app functionality which is what we had in mind with our prompts.

**Conclusion**

Half marks for the LLM here. Took care of FastAPI app no problem but was fairly hopeless at the Optuna objective function